# Simulate current injections into one or more neurons in a network

First we define the network. Generating the network is relatively fast, so you can generate a larger network. Simulating the full network is slower, so by default only the neurons receiving a current injection, and their post synaptic targets are simulated.

In the cell below we define the number of neurons we want, and the volume we place them inside.

In [1]:
import os

n_dSPN = 4
n_iSPN = 4
n_FS = 4
n_LTS = 1
n_ChIN = 0


network_path = os.path.join("networks","pair_recording_simulation")
volume_type="cube"
side_len = 100e-6

Snudda init writes the ```network-config.json``` file.

In [2]:
from snudda.init.init import SnuddaInit
cnc = SnuddaInit(struct_def={}, network_path=network_path, random_seed=1337)
cnc.define_striatum(num_dSPN=n_dSPN, num_iSPN=n_iSPN, num_FS=n_FS, num_LTS=n_LTS, num_ChIN=n_ChIN,
                    volume_type="cube", side_len=side_len)
cnc.write_json()

Using cube for striatum
Adding neurons: FS from dir $SNUDDA_DATA/neurons/striatum/fs
Adding neurons: dSPN from dir $SNUDDA_DATA/neurons/striatum/dspn
Adding neurons: iSPN from dir $SNUDDA_DATA/neurons/striatum/ispn
Checking boundaries, to make sure P is not too high
LTS axon density P([-0.0002],[-0.0001],[-3.e-05]) = [1.84256218e-10]
LTS axon density P([-0.0002],[-0.0001],[3.e-05]) = [1.84256218e-10]
LTS axon density P([-0.0002],[0.0001],[-3.e-05]) = [1.84256218e-10]
LTS axon density P([-0.0002],[0.0001],[3.e-05]) = [1.84256218e-10]
LTS axon density P([0.0009],[-0.0001],[-3.e-05]) = [1.72360142e-23]
LTS axon density P([0.0009],[-0.0001],[3.e-05]) = [1.72360142e-23]
LTS axon density P([0.0009],[0.0001],[-3.e-05]) = [1.72360142e-23]
LTS axon density P([0.0009],[0.0001],[3.e-05]) = [1.72360142e-23]
Adding neurons: LTS from dir $SNUDDA_DATA/neurons/striatum/lts
Writing networks/pair_recording_simulation/network-config.json


Snudda Place, Detect and Prune. Here the neurons are placed in the network, synapse touch detection is done, and finally pruning of the putative synapses.

In [3]:
from snudda.place import SnuddaPlace
sp = SnuddaPlace(network_path=network_path)
sp.place()

In [4]:
from snudda.detect import SnuddaDetect
sd = SnuddaDetect(network_path=network_path)
sd.detect()

Instantiates par_id = 0
morph_tag = lts_morp_2019-11-07_centered_no_axon.swc
No d_view specified, running distribute neurons in serial
Processing hyper voxel : 24/80 (13 neurons)
Processing hyper voxel : 4/80 (13 neurons)
Processing hyper voxel : 5/80 (13 neurons)
Processing hyper voxel : 25/80 (13 neurons)
Processing hyper voxel : 21/80 (10 neurons)
Processing hyper voxel : 29/80 (9 neurons)
Processing hyper voxel : 9/80 (9 neurons)
Processing hyper voxel : 20/80 (9 neurons)
Processing hyper voxel : 28/80 (9 neurons)
Processing hyper voxel : 1/80 (8 neurons)
Processing hyper voxel : 8/80 (8 neurons)
Processing hyper voxel : 0/80 (6 neurons)
Processing hyper voxel : 26/80 (2 neurons)
Processing hyper voxel : 45/80 (2 neurons)
Processing hyper voxel : 22/80 (1 neurons)
Processing hyper voxel : 2/80 (1 neurons)
Processing hyper voxel : 33/80 (1 neurons)
Processing hyper voxel : 44/80 (1 neurons)
Processing hyper voxel : 46/80 (1 neurons)
Processing hyper voxel : 48/80 (1 neurons)
Process

In [ ]:
from snudda.detect import SnuddaPrune
spr = SnuddaPrune(network_path=network_path)
spr.prune()
spr = None

Read 2113 out of total 2113 synapses
Read 62 out of total 62 synapses
> /home/hjorth/HBP/Snudda/snudda/detect/prune.py(1922)prune_synapses_helper()
   1920                     import pdb
   1921                     pdb.set_trace()
-> 1922                     synapses[next_read_pos:read_end_idx, 12] = synapses[next_read_pos, 12]
   1923 
   1924             # Temp check

ipdb> next_read_pos
0
ipdb> read_end_idx
32
ipdb> synapses[next_read_pos:read_end_idx,:]
array([[     2,      0,     93,    137,    100,      5,     10,    259,
            69,     47,    381,   1340,  43005],
       [     2,      0,     94,    138,    100,      5,     10,    230,
            66,     47,    351,   4137, 843288],
       [     2,      0,     95,    138,    100,      5,     10,    229,
            64,     47,    339,   1125,  43898],
       [     2,      0,     95,    139,    100,      5,     10,    225,
            61,     47,    315,    562, 428235],
       [     2,      0,     96,    139,    100,      5

We compile the neuron mechanisms for the simulation. Here we just use the default mechanisms provided.

In [ ]:
!nrnivmodl ../../snudda/data/neurons/mechanisms

Run the NEURON simulation of the network. The setup is defined in the [experiment-config-6.json](../../snudda/data/experiment_config/experiment-config-6.json) file.

In [ ]:
from snudda.simulate.pair_recording import PairRecording 
experiment_config_file = os.path.join("..","..", "snudda", "data", "experiment_config", "experiment-config-6.json")
pr = PairRecording(network_path=network_path, experiment_config_file=experiment_config_file)

In [ ]:
pr.run()

## Plot the results

First we start with an overview trace of all the neurons that receive a current injection, and their post synaptic targets.

In [ ]:
%matplotlib notebook
pr.plot_trace_overview()

We can also plot the individual post synaptic traces, with the current pulses marked.

In [ ]:
pr.plot_traces()

The code below shows how you can use PlotTraces to load the voltage traces and plot the data yourself if you do not have the PairRecording object available anymore.

In [ ]:
%matplotlib notebook
from snudda.plotting import PlotTraces
pt = PlotTraces(file_name=os.path.join(network_path,"simulation","network-voltage-6.txt"),
                network_file=os.path.join(network_path, "network-synapses.hdf5"))

In [ ]:
pt.plot_traces([x for x in pt.voltage])

In [ ]:
pt.plot_traces([11], offset=0)

In [ ]:
pt.plot_traces([12], offset=0)

In [ ]:
pt.plot_traces([x for x in pt.voltage], offset=0)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot([x for x in pr.synapse_currents[0][2]])
plt.show()

In [ ]:
pr.synapse_currents[0][2]

In [ ]:
pr.neurons[1].name